In [107]:
import robotic as ry
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import time
from sklearn.cluster import KMeans

pclColor = [0,0,255]
max_trials = 10
max_vel = 100
max_acc = max_vel

C = ry.Config()
C.addFile('/home/vboxuser/Desktop/CS449/Migi/hw3/GFiles-HW3/environment.g')

# C.addFile('./environment.g')
# C.addFile(ry.raiPath('environment.g'))
# qHome = C.getJointState()
# print(qHome)
C.view()
print(ry.raiPath('environment.g'))

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/robotic/rai-robotModels/environment.g


# 1.1 Perception Module

### Get point clouds for each camera

In [108]:
numOfCameras = 4
cam = [None] * numOfCameras
rgb = [None] * numOfCameras
depth = [None] * numOfCameras
pcl = [None] * numOfCameras


for i in range(numOfCameras):
  cam[i] = ry.CameraView(C)
  cam[i].setCamera('camera'+str(i+1))
  rgb[i], depth[i] = cam[i].computeImageAndDepth(C)
  pcl[i] = ry.depthImage2PointCloud(depth[i], cam[i].getFxycxy())
  print(pcl[i].shape)

(360, 640, 3)
(360, 640, 3)
(360, 640, 3)
(360, 640, 3)


### Now normalise the points relative to world frame

In [109]:
for i in range(numOfCameras):
  curCam = C.getFrame("camera"+str(i+1))  
  curCamRotMat = curCam.getRotationMatrix()
  curCamPos = curCam.getPosition()
    
  for h in range(pcl[i].shape[0]):    # Iterate over height
    for w in range(pcl[i].shape[1]):  # Iterate over width
      point = pcl[i][h, w]            # Get the 3D point [x, y, z]
     
      # Apply rotation
      rotated_point = np.dot(curCamRotMat, point)  # Rotate the point
      
      # Apply translation
      transformed_point = rotated_point + curCamPos  # Add the camera position
      
      pcl[i][h, w] = transformed_point      
      

# # Visualize the point cloud
# pclFrames = [None] * numOfCameras

# for i in range(numOfCameras):
#   pclFrames[i] = C.addFrame('pcl'+str(i+1), 'world')
#   pclFrames[i].setPointCloud(pcl[i], pclColor)
# C.view()

### Now we extract the ones inside the bounding box

In [110]:
centre = C.getFrame("bin").getPosition()
centre[2] = centre[2] + 0.087  # Raise slightly to remove bin surface

off = .1
offsets = [0.3, 0.25, 0.05]

bounding_box_min_x = centre[0] - offsets[0]
bounding_box_max_x = centre[0] + offsets[0]

bounding_box_min_y = centre[1] - offsets[1]
bounding_box_max_y = centre[1] + offsets[1]

bounding_box_min_z = centre[2] - offsets[2]
bounding_box_max_z = centre[2] + offsets[2]


for i in range(numOfCameras):
  for h in range(pcl[i].shape[0]):  
    for w in range(pcl[i].shape[1]):  
      point = pcl[i][h, w] 
      eachX, eachY, eachZ = point  
      
      # Check if the point is within the bounding box
      if not (bounding_box_min_x < eachX < bounding_box_max_x and
              bounding_box_min_y < eachY < bounding_box_max_y and
              bounding_box_min_z < eachZ < bounding_box_max_z):
          pcl[i][h, w] = [0, 0, 0]


### Now combine pcl from the 4 cameras and view the final bounded point cloud

In [111]:
# pclFrames = [None] * numOfCameras

pclCombined = np.concatenate(pcl, axis=0)
pclFrame = C.addFrame('pcl', 'world')
pclFrame.setPointCloud(pclCombined, pclColor)
  
C.view()

0

# 1.2 Prediction Module

In [123]:
voxelSize = 0.008
numOfObjects = 3
degToleration = 10

### Now we do the normals usoing open3d

In [124]:
# Reshape to a 2D array
pclCombined_reshaped = pclCombined.reshape(-1, 3)  # Collapse the first two dimensions into one

print(pclCombined_reshaped.shape)  # Should now be (1440 * 640, 3)

# Convert to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pclCombined_reshaped)

# #  Visualize the point cloud
# o3d.visualization.draw_geometries([pcd])
print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd.voxel_down_sample(voxel_size=voxelSize)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
# o3d.visualization.draw_geometries([downpcd], normals=True)

# # print list of normals
# normals = np.asarray(downpcd.normals)
# print(normals)

(921600, 3)
Downsample the point cloud with a voxel of 0.05


### Now we do the clustering using kmeans from sklearn

In [125]:
# Cluster using kmeans from sklearn, there are 3 objects in the point cloud so we will use 3 clusters
kmeans = KMeans(n_clusters=numOfObjects, random_state=0).fit(np.asarray(downpcd.points))
print(kmeans.labels_)

# Visualize the clusters
clusters = []
for i in range(numOfObjects):
  cluster = downpcd.select_by_index(np.where(kmeans.labels_ == i)[0])
  cluster.paint_uniform_color([(i == 0), (i == 1), (i == 2)])
  clusters.append(cluster)
  
# Check if normals are estimated
if downpcd.has_normals():
    print("Normals are already estimated.")
else:
    print("Normals are not estimated yet. Estimating now...")
    downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    print("Normals estimated.")

# Verify magnitudes of normals
normals = np.asarray(downpcd.normals)
magnitudes = np.linalg.norm(normals, axis=1)
print("Normal magnitudes (first 10):", magnitudes[:10])
print("Are normals normalized? ", np.allclose(magnitudes, 1.0, atol=1e-6))

# # Visualize point cloud with normals
# print("Visualizing the point cloud with normals...")
# o3d.visualization.draw_geometries([downpcd], point_show_normal=True)

/home/vboxuser/Desktop/CS449/rai_venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[0 1 1 ... 0 1 1]
Normals are already estimated.
Normal magnitudes (first 10): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Are normals normalized?  True


In [126]:
# Fix normals orientation by aligning them towards a camera position
def reorient_normals(cluster):
    print("Reorienting normals...")
    viewpoint = np.array([0, 0, 0])  # Replace with the actual camera or world origin
    cluster.orient_normals_towards_camera_location(camera_location=viewpoint)

# Reorient normals for each cluster
for i, cluster in enumerate(clusters):
    print(f"Fixing normals for Cluster {i+1}...")
    reorient_normals(cluster)

# # Visualize the corrected normals
# for i, cluster in enumerate(clusters):
#     print(f"Visualizing corrected normals for Cluster {i+1}...")
#     o3d.visualization.draw_geometries([cluster], point_show_normal=True)


Fixing normals for Cluster 1...
Reorienting normals...
Fixing normals for Cluster 2...
Reorienting normals...
Fixing normals for Cluster 3...
Reorienting normals...


### Now we find antipodal grasp points

In [127]:
def find_antipodal_grasp_points(cluster):
    points = np.asarray(cluster.points)
    normals = np.asarray(cluster.normals)
    
    grasp_pairs = []
    
    # Iterate through all point pairs
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            # Compute the angle between normals
            normal_i = normals[i]
            normal_j = normals[j]
            dot_product = np.dot(normal_i, normal_j)
            angle = np.arccos(dot_product)  # Angle in radians
            
            # Check if angle is close to 180 degrees (antipodal condition)
            if np.abs(np.degrees(angle) - 180) < degToleration:  # 10-degree tolerance
                grasp_pairs.append((i, j))
    
    return grasp_pairs

def visualize_grasp_pairs(cluster, grasp_pairs):
    import open3d as o3d

    points = np.asarray(cluster.points)
    lines = []
    line_colors = []

    # Create lines connecting grasp pairs
    for pair in grasp_pairs:
        idx1, idx2 = pair
        lines.append([idx1, idx2])
        line_colors.append([1, 0, 0])  # Red lines for grasp pairs

    # Create Open3D line set for visualization
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(line_colors)

    # Visualize
    print("Visualizing grasp pairs...")
    o3d.visualization.draw_geometries([cluster, line_set])




# Process each cluster
for i, cluster in enumerate(clusters):
    print(f"Processing Cluster {i+1}...")
    
    grasp_points = find_antipodal_grasp_points(cluster)
    print(f"Number of potential grasp pairs in Cluster {i+1}: {len(grasp_points)}")
    
    print(f"Visualizing grasp pairs for Cluster {i+1}...")
    visualize_grasp_pairs(cluster, grasp_points)
    


Processing Cluster 1...


/tmp/ipykernel_4495/1478565827.py:14: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(dot_product)  # Angle in radians


Number of potential grasp pairs in Cluster 1: 77
Visualizing grasp pairs for Cluster 1...
Visualizing grasp pairs...
Processing Cluster 2...
Number of potential grasp pairs in Cluster 2: 402
Visualizing grasp pairs for Cluster 2...
Visualizing grasp pairs...
Processing Cluster 3...
Number of potential grasp pairs in Cluster 3: 194
Visualizing grasp pairs for Cluster 3...
Visualizing grasp pairs...
